# Minyatür GPT ile metin oluşturma


## Giriş

Model dikkat katmanında, nedensel maskelemeye sahip tek bir Transformer bloğundan oluşur.

Eğitim için IMDB duyarlılık sınıflandırma veri kümesindeki metni kullanacağız.
Belirli bir bilgi istemi için yeni film incelemeleri oluşturacağız.

**Referanslar:**

- [GPT](https://www.semanticscholar.org/paper/Improving-Language-Understanding-by-Generative-Radford/cd18800a0fe0b668a1cc19f2ec95b5003d0a5035)
- [GPT-2](https://www.semanticscholar.org/paper/Language-Models-are-Unsupervised-Multitask-Learners-Radford-Wu/9405cc0d6169988371b2755e573cc28650d14dfe)
- [GPT-3](https://arxiv.org/abs/2005.14165)

## Kurulum

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import numpy as np
import os
import re
import string
import random


## Bir Transformer bloğunu katman olarak uygulama

In [2]:

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Kişisel dikkatte iç çarpım matrisinin üst yarısını maskeleyin.
    Bu, gelecekteki belirteçlerden mevcut belirteçlere bilgi akışını önler.
    1'ler alt üçgende, sağ alt köşeden sayılıyor.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)


## Gömme katmanını uygulama

İki ayrı katıştırma katmanı oluşturun: biri tokenler için ve diğeri tokenlerin dizini için
(pozisyonlar).

In [3]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


## Minyatür GPT modelini uygulama

In [4]:
vocab_size = 20000  # Yalnızca ilk 20 bin kelimeyi dikkate alın
maxlen = 80  # Maksimum dizi boyutu
embed_dim = 256  # Her belirteç için gömme boyutu
num_heads = 2  # Attention başlığı sayısı
feed_forward_dim = 256  # Transformatörün içindeki ileri beslemeli ağda gizli katman boyutu


def create_model():
    inputs = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam", loss=[loss_fn, None],
    )  # Transformatör bloğundan kelime yerleştirmeye dayalı kayıp ve optimizasyon yok
    return model


## Verileri sözcük düzeyinde dil modellemesi için hazırlama

IMDB veri setini indirin ve bir metin için eğitim ve doğrulama setlerini birleştirelim.

In [5]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  68.8M      0  0:00:01  0:00:01 --:--:-- 68.7M


In [6]:

batch_size = 128

# Veri kümesi, her incelemeyi ayrı bir metin dosyasında içerir
# Metin dosyaları dört farklı klasörde bulunur.
# Tüm dosyaların bir listesini oluştur
filenames = []
directories = [
    "aclImdb/train/pos",
    "aclImdb/train/neg",
    "aclImdb/test/pos",
    "aclImdb/test/neg",
]
for dir in directories:
    for f in os.listdir(dir):
        filenames.append(os.path.join(dir, f))

print(f"{len(filenames)} files")

# Metin dosyalarından veri kümesi oluşturma
random.shuffle(filenames)
text_ds = tf.data.TextLineDataset(filenames)
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)


def custom_standardization(input_string):
    """ Html satır sonu etiketlerini kaldırın ve noktalama işaretlerini işleyin """
    lowercased = tf.strings.lower(input_string)
    stripped_html = tf.strings.regex_replace(lowercased, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"([{string.punctuation}])", r" \1")


# Bir vektörleştirme katmanı oluşturun ve bunu metne uyarlayın
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()  # Belirteç dizinlerinden sözcükleri geri almak için


def prepare_lm_inputs_labels(text):
    """
    Sözcük dizilerini 1 konum kaydırarak konum (i) için hedef
    konumdaki kelime (i+1). Model sonraki kelimeyi tahmin etmek için, (i) konumuna kadar tüm kelimeleri kullanacaktır. 
    """
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


text_ds = text_ds.map(prepare_lm_inputs_labels)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)


50000 files


## Metin oluşturmak için bir Keras callback uygulayın

In [7]:

class TextGenerator(keras.callbacks.Callback):
    """Eğitilmiş bir modelden metin oluşturmak için callback adımları:
    1. Modele bazı başlangıç girişleri ile besleyin
    2. Bir sonraki token için olasılıkları tahmin edin
    3. Sonraki tokeni örnekleyin ve sonraki girişe ekleyin

    Arguments:
        max_tokens: Tamsayı değerindedir, inputtan sonra oluşturulacak token sayısı.
        start_tokens: Başlatma tokeni için belirteç dizinleri.
        index_to_word: TextVectorization katmanından elde edilen dizelerin listesi.
        top_k: Tamsayı değerindedir, token tahminlerinden örnek sayısı.
        print_every: Bu kadar epoch'tan sonra yazdır.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"üretilen text:\n{txt}\n")


# Başlatma cümlesini belirt
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

start_prompt = "this movie is"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 40
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)


## Modelimizi eğitelim.

GPU kullanarak eğitmenizi tavsiye ederim.

In [8]:
model = create_model()

model.fit(text_ds, verbose=2, epochs=25, callbacks=[text_gen_callback])

Epoch 1/25
1/1 [==============================] - 0s 21ms/step
üretilen text:
this movie is not an entertaining movie for a good film with a very well , but if you want it to see the film . the characters are the acting , the movie was a good thing i could be disappointed . .

391/391 - 89s - loss: 5.6011 - dense_2_loss: 5.6011 - 89s/epoch - 228ms/step
Epoch 2/25
1/1 [==============================] - 0s 23ms/step
üretilen text:
this movie is a great movie that it makes no sense . i love all , i was looking for the first viewing , and i didn 't expect a movie in it but then . if you have you are the first time

391/391 - 71s - loss: 4.7092 - dense_2_loss: 4.7092 - 71s/epoch - 182ms/step
Epoch 3/25
1/1 [==============================] - 0s 19ms/step
üretilen text:
this movie is one of my favorite shows of the many movies that are not a comedy . there is no one of that will be seen in this film . it is just about a young boy with his wife and a young

391/391 - 73s - loss: 4.4603 - dense